In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import DataLoader, IterableDataset

In [ ]:
class TokenizedExamplesDataset(IterableDataset):
    def __init__(self, examples):
        self.examples = examples

    def __iter__(self):
        return iter(self.examples)

    def __len__(self):
        return len(self.examples)

class InferenceModel:
    def __init__(self, model_path, stride=0.8, use_sliding_window=True, batch_size=1, device="cpu"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForTokenClassification.from_pretrained(model_path).to(device)
        self.model.eval()

        self.use_sliding_window = use_sliding_window
        self.stride = stride
        self.batch_size = batch_size
        self.device = device
        self.fp16 = device != "cpu"

        if self.fp16:
            self._check_fp16_support()

    def _check_fp16_support(self):
        try:
            from torch.cuda import amp
        except ImportError as e:
            print("FP16 not supported: ", e)
            self.fp16 = False

    def prepare_examples(self, texts):
        tokenized_examples = []
        for text in texts:
            # Encode each text to input IDs and attention mask tensors, add special tokens
            encoded_text = self.tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
            tokenized_examples.append(encoded_text)
        return TokenizedExamplesDataset(tokenized_examples)

    def predict(self, texts):
        dataset = self.prepare_examples(texts)
        predictions = []
        for batch in DataLoader(dataset, batch_size=self.batch_size):
            batch = {key: val.to(self.device) for key, val in batch.items()}
            with torch.no_grad(), torch.cuda.amp.autocast(enabled=self.fp16):
                outputs = self.model(**batch)
                logits = outputs[0]
                predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        return predictions

In [ ]:
# Configuration variables
model_path = "path_to_model_directory"
input_file = "path_to_input_file.tsv"
output_file = "path_to_output_file.tsv"

# Instantiate model
model = InferenceModel(model_path)

# Reading and processing input
with open(input_file, 'r') as file:
    texts = [line.strip().split('\t')[1] for line in file if line.strip()]

# Predicting
predictions, _, _ = model.predict(texts)

# Writing predictions to output file
with open(output_file, 'w') as file:
    for pred in predictions:
        file.write(f'{pred}\n')